In [36]:
import GUI
from nltk.stem import PorterStemmer # type: ignore
import numpy as np # type: ignore
import pandas as pd # type: ignore
import re

In [37]:
# function to create the excel files
def create_excel_dataframe(input , name, columns, index):
    df = pd.DataFrame(input, columns = columns, index = index) # Create a pandas dataframe
    df.to_excel(name) # Create an excel file
    return df

In [ ]:
raw_document = GUI.select_file() # Select File GUI implementation

raw_dataset = raw_document # the raw dataset that read from the one file

raw_dataset = raw_dataset.split(".I") # split the docs by index
raw_dataset.remove(raw_dataset[0]) # remove the empty first index

doc_index = 0

# write each index in 'raw_dataset'
for doc in raw_dataset:
    doc_index += 1
    doc_path = "/home/glados/Documents/AmirAli Toori/Lessons/Python/IR-Project/docs" + "/doc" + str(doc_index) + ".txt"
    
    
    with open(doc_path, "w") as document: # write each index into the separated doc
        document.write(doc)


GUI.custom_window("700x35", "Extract Documents", f"There have been {doc_index} documents are extracted.")

In [24]:
# read the list of stop words
stopword_file_path = "/home/glados/Documents/AmirAli Toori/Lessons/Python/IR-Project/gist_stopwords.txt"
with open(stopword_file_path, "r") as file:
    stop_words = file.read()
    
GUI.custom_window("700x35", "Stop words", "Read the stop word list.")


# separate the doc section from the document
def transfer_to_list(doc):
    doc = re.sub(r"[^\w\s]", " ", doc)
    doc = doc.split()
    return doc

def delete_eye(stem_word):
    return stem_word.strip('i')
    

# stemming
stemmer = PorterStemmer()
dictionary = []
def create_dictionary(doc, dictionary):
    for element in doc:
        element = element.lower()
        if element.isalnum() and element not in stop_words:
            stem_word = stemmer.stem(element)
            if stem_word[-1] == 'i':
               stem_word = delete_eye(stem_word)
            if stem_word not in dictionary and stem_word in raw_document:
                dictionary.append(stem_word)


GUI.custom_window("700x35", "Stemming Process", "The stemming process have been successful.")

# read each doc and pass them through the methods
for num in range(1, doc_index + 1):
    doc_path = "/home/glados/Documents/AmirAli Toori/Lessons/Python/IR-Project/docs" + "/doc" + str(num) + ".txt"
    with open(doc_path, "r") as file:
        doc = file.read()
        doc = transfer_to_list(doc)
        create_dictionary(doc, dictionary)

df = pd.DataFrame(dictionary, columns=["Words"])
df


,Words
0,experiment
1,investig
2,aerodynam
3,slipstream
4,brenckman
...,...
6878,1399
6879,wang
6880,a269
6881,kleeman


In [25]:
df.to_excel("dictionary.xlsx")

In [26]:
term_doc_matrix = np.zeros((len(dictionary), doc_index))
tf_array = np.zeros((len(dictionary), doc_index))
idf_array = np.zeros((len(dictionary), 1))
tf_idf_array = np.zeros((len(dictionary), doc_index))

In [27]:
# def term_doc(doc, dictionary, doc_no):
#     for word in dictionary:
#         if word in doc:
#             row = dictionary.index(word)
#             column = doc_no - 1
#             term_doc_matrix[row, column] = 1
    

def calculate_term_frequency(doc, dictionary, doc_no):
    for word in dictionary:
        if word in doc:
            frequency = doc.count(word)
            row = dictionary.index(word)
            column = doc_no - 1
            tf_array[row, column] += frequency


def calculate_inverse_document_frequency(tf_array, idf_array):
    # for word in dictionary:
    #     if word in doc:
    #         row = dictionary.index(word)
    #         idf_array[row, 0] += 1
    
    row, column = np.shape(tf_array)
    for row in tf_array:
        for column in tf_array:
            if tf_array[row, column] > 0:
                idf_array[row, 0] += 1

    return idf_array

In [28]:
doc_names_list = []
for num in range(1, doc_index + 1):
    doc_path = "/home/glados/Documents/AmirAli Toori/Lessons/Python/IR-Project/docs" + "/doc" + str(num) + ".txt"
    doc_names_list.append("doc" + str(num) + ".txt")
    with open(doc_path, "r") as file:
        doc = file.read()
        # term_doc(doc, dictionary, num)
        calculate_term_frequency(doc, dictionary, num)
        # calculate_inverse_document_frequency(doc, dictionary)
    
    calculate_inverse_document_frequency(tf_array, idf_array)

In [29]:
# calculating tf array
row, column = np.shape(tf_array)
for r in range(row):
    for c in range(column):
        if tf_array[r, c] > 0:
            extracted_element = tf_array[r, c]
            tf_array[r, c] = 1 + np.log10(extracted_element)
        else:
            tf_array[r, c] = 0
            
GUI.custom_window("700x35", "Complete!", "The calculation of Term Frequency is have been successful.")

In [30]:
count_of_documents = column
row, column = np.shape(idf_array)


# calculating idf array
for r in range(row):
    if idf_array[r, 0] > 0:
        extracted_element = idf_array[r, 0]
        idf_array[r, 0] = np.log10(count_of_documents / extracted_element)
    else:
        idf_array[r, 0] = 0

GUI.custom_window("700x35", "Complete!", "The calculation of Inverse Document Frequency is have been successful.")

In [31]:
# multiply peer to peer each row of tf-array in idf-array
row, column = np.shape(tf_idf_array)
r, c = 0, 0
while r < row:
        tf_idf_array[r, c]= tf_array[r, c] * idf_array[r, 0]
        c += 1
        if c == column:
            r += 1
            c = 0

In [32]:

tf_dataframe = create_excel_dataframe(tf_array, "tf_excel.xlsx", doc_names_list, dictionary)

idf_dataframe = create_excel_dataframe(idf_array, "idf_excel.xlsx", ["IDF"], dictionary)

tf_idf_dataframe = create_excel_dataframe(tf_idf_array, "tf_idf_excel.xlsx", doc_names_list, dictionary)

GUI.custom_window("700x35", "Excel File", "Creating the Excel file is DONE!")

In [33]:
tf_dataframe

,doc1.txt,doc2.txt,doc3.txt,doc4.txt,doc5.txt,doc6.txt,doc7.txt,doc8.txt,doc9.txt,doc10.txt,...,doc1391.txt,doc1392.txt,doc1393.txt,doc1394.txt,doc1395.txt,doc1396.txt,doc1397.txt,doc1398.txt,doc1399.txt,doc1400.txt
experiment,1.602060,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
investig,1.301030,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.477121,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
aerodynam,1.301030,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
slipstream,1.778151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
brenckman,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
wang,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
a269,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
kleeman,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [34]:
idf_dataframe

,IDF
experiment,0.538673
investig,0.586221
aerodynam,0.876615
slipstream,1.970037
brenckman,3.146128
...,...
1399,3.146128
wang,3.146128
a269,3.146128
kleeman,3.146128


In [35]:
tf_idf_dataframe

,doc1.txt,doc2.txt,doc3.txt,doc4.txt,doc5.txt,doc6.txt,doc7.txt,doc8.txt,doc9.txt,doc10.txt,...,doc1391.txt,doc1392.txt,doc1393.txt,doc1394.txt,doc1395.txt,doc1396.txt,doc1397.txt,doc1398.txt,doc1399.txt,doc1400.txt
experiment,0.862986,0.000000,0.0,0.0,0.000000,0.0,0.538673,0.000000,0.00000,0.0,...,0.000000,0.538673,0.000000,0.0,0.000000,0.538673,0.538673,0.0,0.000000,0.000000
investig,0.762692,0.586221,0.0,0.0,0.000000,0.0,0.586221,0.586221,0.86592,0.0,...,0.000000,0.000000,0.586221,0.0,0.586221,0.000000,0.000000,0.0,0.000000,0.586221
aerodynam,1.140503,0.000000,0.0,0.0,0.876615,0.0,0.000000,0.000000,0.00000,0.0,...,0.876615,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
slipstream,3.503023,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
brenckman,3.146128,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,3.146128,0.000000
wang,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,3.146128,0.000000
a269,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,3.146128,0.000000
kleeman,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,3.146128
